# Implementation of the `scaredyFish` model in Julia
### First step
Let's first define a spatial landscape and set parameter values

In [14]:
## ---
## init
## ---
using Distributions, Distances, IterTools, BenchmarkTools


## -
## Define my landscape
## -

## number of rows and cols
nrows = 10
ncols = 100

## landscape is a binary matrix, where 1s are foraging patches (white tiles)
landscape = rand( Bernoulli( 0.5 ), nrows, ncols )

## indices for white tiles
whiteTiles = getindex.( findall( y -> y == 1, landscape ), [ 1 2 ] )


## -
## Define a bunch of parameters
## -

## number of agents
agents = 100

## time steps
maxtime = 100

## walk step, this is the SD of the normal distribution from which walk step lengths are drawn
walk = 0.05

## run step, this is the SD of the normal distribution from which run step lengths are drawn
run = 0.2

## strength of drift back toward white tile if you're in a black tile
drift = 0.2

## excitation strength
strengthExcitation = 5

## inhibition strength
strengthInhibition = 0.25

## sensory length scale radius
sensory = 2.5

## decision threshold
threshold = 1

## trigger time for false startle
trigger = 5;

### Second step
Define a function that will loop through `times` and `agents` and do all the calculations

In [15]:
## ---
## Function to loop through time steps
## ---
function scaredyFish( )
    
    ## -
    ## Prep simulation, define some empty arrays to fill in
    ## -

    ## empty 3D array to populate ( time x number of variables x agents )
    ## my columns are 
    ## | 1. time 
    ## | 2. agent id 
    ## | 3. x 
    ## | 4. y 
    ## | 5. state 0 (feed) or 1 (flee)
    ## | 6. in a white tile or not
    r = zeros( maxtime, 6, agents );

    ## populate first row of each matrix
    for agent in 1 : size( r, 3 )

        ## put a random number in the first row
        r[ 1, 1, agent ] = 1
        r[ 1, 2, agent ] = agent
        
        ## sample a white tile
        sampledTile = whiteTiles[ sample( 1 : size( whiteTiles )[ 1 ], 1 ), : ]
        
        ## put a random number in the first row
        r[ 1, 3, agent ] = sampledTile[ 1 ] + Float64.( rand( Uniform( -0.49, 0.49 ) ) )
        r[ 1, 4, agent ] = sampledTile[ 2 ] + Float64.( rand( Uniform( -0.49, 0.49 ) ) )
        
        ## fill in state and location
        r[ 1, 5, agent ] = 0
        r[ 1, 6, agent ] = 1

    end
    
    
    ## -
    ## Start workhorse
    ## -
    
    ## for t in ___time___
    for t in 2 : maxtime

        ## for a in ___agents____
        for a in 1 : agents
            
            ## add info to table
            r[ t, 1, a ] = t

            ## add info to table
            r[ t, 2, a ] = a
            
            ## -
            ## first we need to know if we're in a white tile or black tile, and where the nearest white tile is
            ## -

            ## grab my current x and y before updating
            currX = r[ t - 1, 3, a ]
            currY = r[ t - 1, 4, a ]

            ## round them to find the nearest tile
            currPatchX = round.( Int, currX )
            currPatchY = round.( Int, currY )

            ## check that it is actually a tile, and you are not out of bounds
            if ( 0.5 < currPatchX < ( 0.5 + size( landscape, 1 ) ) ) && ( 0.5 < currPatchY < ( 0.5 + size( landscape, 2 ) ) )

                ## note that I am in bounds
                inBounds = 1

                ## if this is a white tile, we are already done
                if landscape[ currPatchX, currPatchY ] == 1

                    ## make this my home patch
                    homePatchX = currPatchX
                    homePatchY = currPatchY

                    ## record status in output ( 1 if I'm in a white tile, 0 if not )
                    r[ t, 6, a ] = 1

                ## otherwise, find the nearest white tile
                else

                    ## distances to white tiles
                    toWhiteTiles = colwise( Euclidean(), [ currX, currY ], transpose( whiteTiles ) )

                    ## which min
                    homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                    homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                    ## record status in output
                    r[ t, 6, a ] = 0

                end

            ## if I'm out of bounds...
            else

                ## note it
                inBounds = 0

                ## distances to white tiles
                toWhiteTiles = colwise( Euclidean(), [ currX, currY ], transpose( whiteTiles ) )

                ## which min
                homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                ## record status in output
                r[ t, 6, a ] = 0

            end


            ## -
            ## Here starts the step updating section
            ## -

            ## if feeding and in bounds
            if r[ t - 1, 5, a ] == 0 && inBounds == 1

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( 0, walk ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( 0, walk ) )

            end

            ## if feeding and out of bounds
            if r[ t - 1, 5, a ] == 0 && inBounds == 0

                ## bias my walk in this direction
                xBias = ( homePatchX - r[ t - 1, 3, a ] )
                yBias = ( homePatchY - r[ t - 1, 4, a ] )

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( xBias * drift, walk ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( yBias * drift, walk ) )

            end

            ## if starting fleeing
            if r[ t - 1, 5, a ] == 1 && r[ t - 2, 5, a ] == 0

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( 0, run ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( 0, run ) )

            end
            
            ## if continuing fleeing
            if r[ t - 1, 5, a ] == 1 && r[ t - 2, 5, a ] == 1

                ## bias my walk in this direction
                xBias = ( r[ t - 1, 3, a ] - r[ t - 2, 3, a ] )
                yBias = ( r[ t - 1, 4, a ] - r[ t - 2, 4, a ] )
                
                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( xBias, 0.005 ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( yBias, 0.005 ) )

            end

        end

        
        ## -
        ## Here starts the information updating section
        ## -

        ## extract current coordinates
        C = hcat( r[ t, 3, : ], r[ t, 4, : ] )

        ## calculate distances between agents
        D = pairwise( Euclidean(), C; dims = 1 )

        ## threshold this matrix
        D[ D .> sensory ] .= 0

        ## convert to 1 / D ^ 2
        D[ D .> 0 ] = 1 ./ D[ D .> 0 ] .^ 2
        
        ##
        ## cycle through agents and calculate excitation and inhibition
        ##
        
        ## grab the states at time t - 1
        currStates = r[ t - 1, 5, : ]

        ## for a in ___agents____
        for a in 1 : agents
           
            ## excitation information
            excitation = strengthExcitation * sum( D[ a, findall( currStates .== 1 ) ] )
            
            ## inhibition information
            inhibition = strengthInhibition * sum( D[ a, findall( currStates .== 0 ) ] )
            
            ## decision making function
            decision = excitation - inhibition
            
            ## if we pass threshold
            if decision > 1
                
                ## update state
                r[ t, 5, a ] = 1
                
            end
            
            ## force the agent back to feed state if its been fleeing for 2 timesteps
            if r[ t - 1, 5, a ] == 1 && r[ t - 2, 5, a ] == 1
                
                ## update state
                r[ t, 5, a ] = 0
                
            end
            
        end
        
        ## -
        ## trigger false alarm
        ## -
        
        ## initiate
        if t == trigger

            ## switch state
            r[ t, 5, sample( 1 : agents, 1 ) ] .= 1

        end

    end
    
    ## return results
    return r
    
    end;

In [13]:
## run my simulation once
r = scaredyFish()

100×6×10 Array{Float64,3}:
[:, :, 1] =
   1.0  1.0  8.63127  69.5215  0.0  1.0
   2.0  1.0  8.63169  69.5465  0.0  1.0
   3.0  1.0  8.65426  69.5511  0.0  1.0
   4.0  1.0  8.76374  69.4788  0.0  1.0
   5.0  1.0  8.80141  69.5189  0.0  0.0
   6.0  1.0  8.75086  69.5808  0.0  1.0
   7.0  1.0  8.80074  69.5686  0.0  1.0
   8.0  1.0  8.77669  69.6179  0.0  1.0
   9.0  1.0  8.76737  69.7091  0.0  1.0
  10.0  1.0  8.78166  69.7094  0.0  1.0
  11.0  1.0  8.80291  69.7268  0.0  1.0
  12.0  1.0  8.75161  69.7268  0.0  1.0
  13.0  1.0  8.71598  69.7203  0.0  1.0
   ⋮                                ⋮
  89.0  1.0  8.90518  68.8036  0.0  0.0
  90.0  1.0  8.92982  68.7229  0.0  0.0
  91.0  1.0  8.94058  68.7031  0.0  0.0
  92.0  1.0  8.91793  68.7394  0.0  0.0
  93.0  1.0  8.99687  68.674   0.0  0.0
  94.0  1.0  9.01914  68.6418  0.0  0.0
  95.0  1.0  9.02811  68.6244  0.0  0.0
  96.0  1.0  8.99252  68.686   0.0  0.0
  97.0  1.0  8.91908  68.6578  0.0  0.0
  98.0  1.0  8.98542  68.6812  0.0  0.0
  9